## Importing libraries

In [1]:
from pandas import DataFrame
import pandas as pd
import json
from user_agents import parse
from urllib.parse import urlparse
pd.options.mode.chained_assignment = None


### Reading file, renaming columns, and droping null values

In [2]:
df= pd.read_json('usa.gov_click_data_3.json',lines = True)
df.rename(columns={'a':'webbrowser_os','tz':'Time Zone','r':'From URL','u':'To URL','t':'Time In','hc':'Time Out','cy':'City','ll':'longitude_latitude'},inplace=True)
df=df[['webbrowser_os','Time Zone','From URL','To URL','Time In','Time Out','City','longitude_latitude']]
df.dropna(axis = 0 , inplace=True )

### Adding new columns that will replace the user agent column

In [3]:
df["Device"]=""
df["System"]=""
df["Browser"]=""
df = df[[ 'Device', 'System', 'Browser', 'webbrowser_os', 'Time Zone', 'From URL', 'To URL', 'Time In', 'Time Out', 'City', 'longitude_latitude']]


### Extracting the wanted information from the useragent and droppint it

In [4]:
for i in df.index:
    ua_string = parse(df['webbrowser_os'][i])
    x = str(ua_string).replace(" ", "").split("/")
    df["Device"][i] = x[0]
    df["System"][i] = x[1]
    df["Browser"][i] = x[2]
df.drop(['webbrowser_os'], axis=1, inplace=True)

### Splitting Longitude and Latitude values

In [5]:
df['longitude_latitude'] = df['longitude_latitude'].astype('str')
df[['Longitude','Latitude']]= df['longitude_latitude'].str.strip('[]').str.split(',',expand=True)
df = df[[ 'Device', 'System', 'Browser', 'Longitude', 'Latitude', 'City',  'From URL', 'To URL', 'Time In', 'Time Out','Time Zone']]

### Transform URLs in short form

In [6]:
for i in df.index:
    df['From URL'][i] = urlparse(df['From URL'][i]).netloc
    df['To URL'][i] = urlparse(df['To URL'][i]).netloc


In [7]:
df['From URL'] = df['From URL'].replace("", "Direct Link")
df['To URL'] = df['To URL'].replace("", "Direct Link")


In [8]:
df.head()

,Device,System,Browser,Longitude,Latitude,City,From URL,To URL,Time In,Time Out,Time Zone
0,PC,Windows7,Chrome17.0.963,42.576698,-70.954903,Danvers,www.facebook.com,www.ncbi.nlm.nih.gov,1.331923e+09,1.331823e+09,America/New_York
1,Other,Other,Other,40.218102,-111.613297,Provo,www.AwareMap.com,www.monroecounty.gov,1.331923e+09,1.308262e+09,America/Denver
2,PC,Windows7,IE8.0,38.9007,-77.043098,Washington,t.co,boxer.senate.gov,1.331923e+09,1.331920e+09,America/New_York
3,PC,MacOSX10.6.8,Safari5.1.2,-23.549999,-46.616699,Braz,Direct Link,apod.nasa.gov,1.331923e+09,1.331923e+09,America/Sao_Paulo
4,PC,Windows7,Chrome17.0.963,42.286499,-71.714699,Shrewsbury,www.shrewsbury-ma.gov,www.shrewsbury-ma.gov,1.331923e+09,1.273672e+09,America/New_York


In [39]:
df.isnull().sum()

Device       0
System       0
Browser      0
Longitude    0
Latitude     0
City         0
From URL     0
To URL       0
Time In      0
Time Out     0
Time Zone    0
dtype: int64

# Saving The Data Frame to A CSV File

In [49]:
df.to_csv(r'C:\ITI\Python\Task 2\target\usa.gov_click_data_3.csv', index=False)
